In [1]:
import numpy as np
import pandas as pd
import pickle
import math
from itertools import product 

NOTE:  
% of biodiesel vs. diesel is calculated from raw data source: "https://pathways-trajectoires.ca/" -> "iet-data-2023-10-10_QConly_BAU_vs_NZ50.zip" for QC:   
- 
ssp245mappingg:  "BAU-Energy consumed by source-data" 
-  ss126 mapping:g "NZ50-Energy consumed by source-data"
- ssp119 /  ssp585  are then interpolate 5

### read in the QC scenario dat, change raw MJ-based % -> kg-based 
 
- heating value per ecoinvent:
    -  diesel, low-sulfur' is a fossil fuel with a calorific value of <b>42.8 MJ/kg </b> https://ecoquery.ecoinvent.org/3.10/cutoff/dataset/6730/documentatio#    
- Biodiesel MJ/kg value not present in premise, assuming <b> 39.9 MJ/kg </b>, averaging from "Table 2: 39.2–40.6 MJ/kg" https://www.sciencedirect.com/science/article/pii/S0301421507001516  

In [2]:
d_bau = pd.read_excel("../BAU-Energy consumed by source-data.xlsx", skiprows=8, header=0 )  
d_nz50 = pd.read_excel("../NZ50-Energy consumed by source-data.xlsx", skiprows=8, header=0 )  

In [3]:
year = [2030, 2040, 2050]
bau_bds_per_bds_ds, bau_ds_per_bds_ds = [] , []

for y in year: 
    b_ds = d_bau[ (d_bau["Period"] == y) & 	(d_bau["Dimension 2"] == "Biodiesel") ]
    ds = d_bau[ (d_bau["Period"] == y) & 	(d_bau["Dimension 2"] == "Diesel") ]
    r_ds = d_bau[ (d_bau["Period"] == y) & 	(d_bau["Dimension 2"] == "Renewable diesel") ]
    
    if len(r_ds) == len(b_ds) == len(ds) == 1: 
        all_d = r_ds["Value"].values[0] +  ds["Value"].values[0] + b_ds["Value"].values[0]
        pert_bds_per_bds_ds =  b_ds["Value"].values[0] / (ds["Value"].values[0] + b_ds["Value"].values[0]) 
        pert_ds_per_bds_ds =  ds["Value"].values[0] / (ds["Value"].values[0] + b_ds["Value"].values[0]) 
        print(f"for year {y}, under BAU, % of diesel and biodiesel in Quebec are : {pert_ds_per_bds_ds}, {pert_bds_per_bds_ds}")
        bau_bds_per_bds_ds.append(round(pert_bds_per_bds_ds,3)) 
        bau_ds_per_bds_ds.append(round(pert_ds_per_bds_ds,3)) 
        
print("we'll assume same market % used for HDV truck in our case as we have no data breaking down by sector")

for year 2030, under BAU, % of diesel and biodiesel in Quebec are : 0.9557991815794296, 0.04420081842057045
for year 2040, under BAU, % of diesel and biodiesel in Quebec are : 0.962624397438007, 0.03737560256199304
for year 2050, under BAU, % of diesel and biodiesel in Quebec are : 0.9505902132762716, 0.049409786723728456
we'll assume same market % used for HDV truck in our case as we have no data breaking down by sector


In [4]:
bau_bds_per_bds_ds

[0.044, 0.037, 0.049]

In [5]:
year = [ 2030, 2040, 2050]
nz50_bds_per_bds_ds = []
nz50_ds_per_bds_ds = []

for y in year: 
    b_ds = d_nz50[ (d_nz50["Period"] == y) & 	(d_nz50["Dimension 2"] == "Biodiesel") ]
    ds = d_nz50[ (d_nz50["Period"] == y) & 	(d_nz50["Dimension 2"] == "Diesel") ]
    r_ds = d_nz50[ (d_nz50["Period"] == y) & 	(d_nz50["Dimension 2"] == "Renewable diesel") ]
    
    if len(r_ds) == len(b_ds) == len(ds) == 1: 
        all_d = r_ds["Value"].values[0] +  ds["Value"].values[0] + b_ds["Value"].values[0]
        pert_bds_per_bds_ds =  b_ds["Value"].values[0] / (ds["Value"].values[0] + b_ds["Value"].values[0]) 
        pert_ds_per_bds_ds =  ds["Value"].values[0] / (ds["Value"].values[0] + b_ds["Value"].values[0]) 
        print(f"for year {y}, under NZ2050, % of diesel and biodiesel are: {pert_ds_per_bds_ds}, {pert_bds_per_bds_ds}")
        nz50_bds_per_bds_ds.append(round(pert_bds_per_bds_ds,3)) 
        nz50_ds_per_bds_ds.append(round(pert_ds_per_bds_ds,3)) 

for year 2030, under NZ2050, % of diesel and biodiesel are: 0.9556895834571276, 0.044310416542872426
for year 2040, under NZ2050, % of diesel and biodiesel are: 0.9383833228643216, 0.061616677135678394
for year 2050, under NZ2050, % of diesel and biodiesel are: 0.8665049074256236, 0.13349509257437647


## 0. convert market% by MJ to market% by kg, as all UPR product flow unit is in kg

In [6]:
bau_ds_kg, bau_bds_kg  = [num / 42.8  for num in bau_ds_per_bds_ds], \
                         [num / 39.9 for num in bau_bds_per_bds_ds]

sum_list = [sum(x) for x in zip(bau_ds_kg, bau_bds_kg)]
#print(sum_list)

bau_ds_perc, bau_bds_perc  =   [x / y for x, y in zip(bau_ds_kg, sum_list)], [x / y for x, y in zip(bau_bds_kg, sum_list)] 

for i in range(len(bau_ds_perc)): 
    print(f"for year {year[i]}, under BAU, % by kg of conv_diesel, biodiesel are: {bau_ds_perc[i]}, {bau_bds_perc[i]} ")

for year 2030, under BAU, % by kg of conv_diesel, biodiesel are: 0.9529524628006675, 0.047047537199332454 
for year 2040, under BAU, % by kg of conv_diesel, biodiesel are: 0.9604172238566461, 0.039582776143353836 
for year 2050, under BAU, % by kg of conv_diesel, biodiesel are: 0.9476251245564045, 0.05237487544359562 


In [7]:
nz50_ds_kg, nz50_bds_kg  = [num / 42.8  for num in nz50_ds_per_bds_ds], \
                         [num / 39.9 for num in nz50_bds_per_bds_ds]

sum_list = [sum(x) for x in zip(nz50_ds_kg, nz50_bds_kg)]
#print(sum_list)

nz50_ds_perc, nz50_bds_perc  =   [x / y for x, y in zip(nz50_ds_kg, sum_list)], [x / y for x, y in zip(nz50_bds_kg, sum_list)] 

for i in range(len(nz50_ds_perc)): 
    print(f"for year {year[i]}, under NZ50, % by kg of conv_diesel, biodiesel are: {nz50_ds_perc[i]}, {nz50_bds_perc[i]} ")

for year 2030, under NZ50, % by kg of conv_diesel, biodiesel are: 0.9529524628006675, 0.047047537199332454 
for year 2040, under NZ50, % by kg of conv_diesel, biodiesel are: 0.9337920847908422, 0.06620791520915774 
for year 2050, under NZ50, % by kg of conv_diesel, biodiesel are: 0.8586992406734896, 0.1413007593265104 


In [8]:
baulist = [bau_ds_perc, bau_bds_perc ]
nzlist = [nz50_ds_perc, nz50_bds_perc]

### 1. we have BAU -> SSP245 and NZ2050 -> SSP126 ,  then linear interpolate for SSP126 and SSP585


In [9]:
def diesel_between_ssp_int(bau_ssp245_bd = bau_bds_per_bds_ds, nz50_ssp126_bd = nz50_bds_per_bds_ds):
    """
    interpolate for SSP119 and SSP585 now from BAU and NZ50 value 
    for simplicity, SSP119 == SSP126 SSP585 == SSP245 except for the 2050 value
    """

    # SSP119: assuming same as SSP126 for all years except 2050 
    ssp119_bds_per_bds_ds = [0,0,0 ]
    ssp119_bds_per_bds_ds[0:2] = nz50_ssp126_bd[0:2]
    ssp119_bds_per_bds_ds[2] = nz50_ssp126_bd[2]*1.1 # for 2050, assuming 10% more biodiesel in 2050 than SSP126
        
    # diesel % is 1 -  bd% for each year then    
    ssp119_ds_per_bds_ds =  [(val1 - val2)   for val1, val2 in zip([1,1,1,1], ssp119_bds_per_bds_ds)]

    # SSP585: assuming same as SSP245 for year2020/2030/2040, then 4th element Y2050 same as 2040:  
    ssp585_bds_per_bds_ds = [0,0,0 ]
    ssp585_bds_per_bds_ds[0:2] = bau_ssp245_bd[0:2]
    ssp585_bds_per_bds_ds[2] = ssp585_bds_per_bds_ds[1]

        
    ssp585_ds_per_bds_ds =   [(val1 - val2)   for val1, val2 in zip([1,1,1,1], ssp585_bds_per_bds_ds)]
    print(f"the SSP119 2030/40/50 biodiesel% and diesel% is: {ssp119_bds_per_bds_ds}, {ssp119_ds_per_bds_ds} " )
    print(f"the SSP585 2030/40/50 biodiesel% and diesel% is: {ssp585_bds_per_bds_ds}, {ssp585_ds_per_bds_ds} " )
  
    return (ssp119_bds_per_bds_ds, ssp119_ds_per_bds_ds), (ssp585_bds_per_bds_ds ,ssp585_ds_per_bds_ds)

In [10]:
ssp119bdandd, ssp585bdandd = diesel_between_ssp_int()

the SSP119 2030/40/50 biodiesel% and diesel% is: [0.044, 0.062, 0.1463], [0.956, 0.938, 0.8537] 
the SSP585 2030/40/50 biodiesel% and diesel% is: [0.044, 0.037, 0.037], [0.956, 0.963, 0.963] 


### nz50 -> mapped to SSP126, and BAU -> mapped to SSP245

In [11]:
bd_allssp_4year = (ssp119bdandd[0], nz50_bds_per_bds_ds, bau_bds_per_bds_ds , ssp585bdandd[0])
bd_allssp_4year

([0.044, 0.062, 0.1463],
 [0.044, 0.062, 0.133],
 [0.044, 0.037, 0.049],
 [0.044, 0.037, 0.037])

In [12]:
d_allssp_4year = (ssp119bdandd[1] , nz50_ds_per_bds_ds , bau_ds_per_bds_ds , ssp585bdandd[1])
d_allssp_4year

([0.956, 0.938, 0.8537],
 [0.956, 0.938, 0.867],
 [0.956, 0.963, 0.951],
 [0.956, 0.963, 0.963])

## 2. now we have BAU -> SSP245, NZ2050 -> SSP119, SSP126 and SSP585 for year 2020/30/40/50, we need interpolate 2025/2035/2045 for each SSP, as five-year gap is needed in tech_P file 

In [13]:
def diesel_between_yr_linear_int(elements):
    interpolated_values = []
    for i in range(len(elements) - 1):
        start = elements[i]
        end = elements[i + 1]
        interpolated_value = round((start + end) / 2 , 3)  # Calculate the interpolated value
        interpolated_values.append(start)  # Add the starting element
        interpolated_values.append(interpolated_value)  # Add the interpolated value
    interpolated_values.append(elements[-1])  # Add the ending element
    return interpolated_values


In [14]:
# four empty list to store 4 SSPx data for years to be interpolated
final_d = [[],[],[],[]]
for i in range(len(d_allssp_4year)):     
    final_d[i] = diesel_between_yr_linear_int(d_allssp_4year[i])

#print(f"the SSP119, 126, 245, 585 biodiesel% at year  2030, 2035, 2040, 2045, 2050 are: \n , {final_d}")

In [15]:
final_d

[[0.956, 0.947, 0.938, 0.896, 0.8537],
 [0.956, 0.947, 0.938, 0.902, 0.867],
 [0.956, 0.96, 0.963, 0.957, 0.951],
 [0.956, 0.96, 0.963, 0.963, 0.963]]

In [16]:
final_bd = [[],[],[],[]]
for i in range(len(bd_allssp_4year)):     
    final_bd[i] = diesel_between_yr_linear_int(bd_allssp_4year[i])

#print(f"the SSP119, 126, 245, 585 biodiesel% at year 2020, 2025 , 2030, 2035, 2040, 2045, 2050 are: \n , {final_d}")

In [17]:
final_bd

[[0.044, 0.053, 0.062, 0.104, 0.1463],
 [0.044, 0.053, 0.062, 0.098, 0.133],
 [0.044, 0.04, 0.037, 0.043, 0.049],
 [0.044, 0.04, 0.037, 0.037, 0.037]]

In [18]:
# assigned year as col names
columns = [ 2030, 2035, 2040, 2045, 2050]
# assigned row index
index = ['SSP119', 'SSP126', 'SSP245', 'SSP585']

final_d_df = pd.DataFrame(final_d, columns=columns, index=index)
final_d_df

2030   2035   2040   2045    2050
SSP119  0.956  0.947  0.938  0.896  0.8537
SSP126  0.956  0.947  0.938  0.902  0.8670
SSP245  0.956  0.960  0.963  0.957  0.9510
SSP585  0.956  0.960  0.963  0.963  0.9630

In [19]:
final_bd_df = pd.DataFrame(final_bd, columns=columns, index=index)
final_bd_df

2030   2035   2040   2045    2050
SSP119  0.044  0.053  0.062  0.104  0.1463
SSP126  0.044  0.053  0.062  0.098  0.1330
SSP245  0.044  0.040  0.037  0.043  0.0490
SSP585  0.044  0.040  0.037  0.037  0.0370

In [20]:
final_bd_df.to_excel("biodiesel_pect_perBDandD.xlsx")
final_d_df.to_excel("diesel_pect_perBDandD.xlsx")

### 3. as we have four different feedstocks for making biodiesel, simply average them in the tech_parameter file for biodiesel